In [1]:
import pandas as pd
from pathlib import Path

base_dir = Path(r"I:\NTS\classified builds")

In [2]:
v10 = pd.read_csv(base_dir / "cb_tfn_v10.csv", low_memory=False)

In [ ]:
edges = [0, 10, 25, 50, 100, 200, 1000]
labels = ['0-10', "'10-25",'25-50','50-100','100-200','200+']
v10['dist_bands'] = pd.cut(v10.TripDisIncSW, bins=edges, labels=labels)

def tld(cb: pd.DataFrame, bins: list[int], group: bool = False):
    """
    Reads in a classified build and bins and bins distance data

    Summary
    -------
        cb (pd.DataFrame): A NTS classified build
        bins (list[int]): a list of ints for distance bins
        group (bool, optional): Whether or not to group the output by the bins and
            sum weighted trips.

    Returns:
        pd.DataFrame: Either the input df with a dist_bins column, or grouped for tlds
    """
    labels = [f"{edges[i]}-{edges[i+1]}" for i in range(len(edges) - 1)]
    cb['dist_bands'] = pd.cut(cb.TripDisIncSW, bins=bins, labels=labels)
    if group:
        return cb.groupby('dist_bands')['weighted_trips'].sum()
    return cb

In [9]:
oa_at = pd.read_csv(r"I:\NTS\area type from postcode\oa_to_at_lookup.csv", index_col=0)
ps_to_oa = pd.read_csv(r"I:\NorMITs Land Use\import\postcode_sector_to_oa.csv", index_col=0)

In [3]:
v10.groupby('main_mode')['weighted_trips'].sum()

main_mode
1    1.527970e+06
2    9.674463e+04
3    3.838119e+06
4    9.799986e+04
5    4.161575e+05
6    1.105870e+05
7    6.829839e+04
8    8.867291e+02
Name: weighted_trips, dtype: float64

In [5]:
v10.IndividualID.nunique()

304945

In [7]:
v10.columns

Index(['PSUID', 'PSUPSect', 'PSUPopDensity', 'PSUAreaType1_B01ID',
       'PSUAreaType2_B01ID', 'PSUGOR_B02ID', 'PSUCounty_B01ID', 'SurveyYear',
       'HouseholdID', 'HHoldOSWard_B01ID', 'HHoldOSLAUA_B01ID',
       'HHoldCounty_B01ID', 'HHoldGOR_B02ID', 'HHoldAreaType1_B01ID',
       'HHoldAreaType2_B01ID', 'HHoldNumAdults', 'NumCarVan_B02ID',
       'NumCarVan', 'W1', 'W2', 'W3', 'IndividualID', 'Age_B01ID', 'Sex_B01ID',
       'XSOC2000_B02ID', 'NSSec_B03ID', 'SIC2007_B02ID', 'CarAccess_B01ID',
       'DrivLic_B02ID', 'EcoStat_B01ID', 'EcoStat_B02ID', 'DayID', 'TravDay',
       'TravelWeekDay_B01ID', 'TravelWeekDay_B02ID', 'TripID', 'JJXSC',
       'JOTXSC', 'JTTXSC', 'JD', 'MainMode_B03ID', 'MainMode_B04ID',
       'MainMode_B11ID', 'TripPurpFrom_B01ID', 'TripPurpTo_B01ID',
       'TripStart_B01ID', 'TripEnd_B01ID', 'TripDisIncSW',
       'TripDisIncSW_B01ID', 'TripTravTime', 'TripOrigCounty_B01ID',
       'TripDestCounty_B01ID', 'TripDestUA2009_B01ID', 'TripOrigUA2009_B01ID',
    

In [10]:
ps_to_oa.set_index('OA11CD', inplace=True)

In [11]:
group_1 = oa_at.groupby(['oa_id', 'msoa_at_id']).sum()['oa_to_msoa_at']
group_2 = group_1.reset_index().groupby('oa_id').max()

In [12]:
df = ps_to_oa.join(group_2)

In [13]:
new = df.groupby(['PCDS11CD', 'msoa_at_id']).count()

In [67]:
new.to_csv(r"I:\NTS\lookups\PC_to_at.csv")

In [14]:
new.reset_index().groupby('PCDS11CD').max()

,msoa_at_id,MSOA11CD,TFN_AT,oa_to_msoa_at
PCDS11CD,,,,
AL1 1,Urban Large (within urban area with pop 100-250k),0,0,18
AL1 2,Urban Large (within urban area with pop 100-250k),0,0,20
AL1 3,Urban Large (within urban area with pop 100-250k),0,0,14
AL1 4,Urban Large (within urban area with pop 100-250k),0,0,27
AL1 5,Urban Large (within urban area with pop 100-250k),0,0,30
...,...,...,...,...
YO8 4,Urban Medium (within urban area with pop 25-100k),0,0,25
YO8 5,Urban Medium (within urban area with pop 25-100k),0,0,14
YO8 6,Rural village and dispersed,0,0,10


In [3]:
v10.columns

Index(['PSUID', 'PSUPSect', 'PSUPopDensity', 'PSUAreaType1_B01ID',
       'PSUAreaType2_B01ID', 'PSUGOR_B02ID', 'PSUCounty_B01ID', 'SurveyYear',
       'HouseholdID', 'HHoldOSWard_B01ID', 'HHoldOSLAUA_B01ID',
       'HHoldCounty_B01ID', 'HHoldGOR_B02ID', 'HHoldAreaType1_B01ID',
       'HHoldAreaType2_B01ID', 'HHoldNumAdults', 'NumCarVan_B02ID',
       'NumCarVan', 'W1', 'W2', 'W3', 'IndividualID', 'Age_B01ID', 'Sex_B01ID',
       'XSOC2000_B02ID', 'NSSec_B03ID', 'SIC2007_B02ID', 'CarAccess_B01ID',
       'DrivLic_B02ID', 'EcoStat_B01ID', 'EcoStat_B02ID', 'DayID', 'TravDay',
       'TravelWeekDay_B01ID', 'TravelWeekDay_B02ID', 'TripID', 'JJXSC',
       'JOTXSC', 'JTTXSC', 'JD', 'MainMode_B03ID', 'MainMode_B04ID',
       'MainMode_B11ID', 'TripPurpFrom_B01ID', 'TripPurpTo_B01ID',
       'TripStart_B01ID', 'TripEnd_B01ID', 'TripDisIncSW',
       'TripDisIncSW_B01ID', 'TripTravTime', 'TripOrigCounty_B01ID',
       'TripDestCounty_B01ID', 'TripDestUA2009_B01ID', 'TripOrigUA2009_B01ID',
    

In [5]:
edges = [0, 10, 25, 50, 100, 200, 1000]
labels = ['0-10', "'10-25",'25-50','50-100','100-200','200+']
v10['dist_bands'] = pd.cut(v10.TripDisIncSW, bins=edges, labels=labels)

def tld(cb: pd.DataFrame, bins: list[int], group: bool = False):
    """
    Reads in a classified build and bins and bins distance data

    Summary
    -------
        cb (pd.DataFrame): A NTS classified build
        bins (list[int]): a list of ints for distance bins
        group (bool, optional): Whether or not to group the output by the bins and
            sum weighted trips.

    Returns:
        pd.DataFrame: Either the input df with a dist_bins column, or grouped for tlds
    """
    labels = [f"{edges[i]}-{edges[i+1]}" for i in range(len(edges) - 1)]
    cb['dist_bands'] = pd.cut(cb.TripDisIncSW, bins=bins, labels=labels)
    if group:
        return cb.groupby('dist_bands')['weighted_trips'].sum()
    return cb

In [7]:
tld(v10, edges, True)

dist_bands
0-10        5.264722e+06
10-25       6.070076e+05
25-50       1.725403e+05
50-100      7.152450e+04
100-200     3.136247e+04
200-1000    9.606159e+03
Name: weighted_trips, dtype: float64

In [16]:
modes = {1: 'Walk',
         2: 'Walk',
         3: 'Cycle',
         4: 'Car Driver',
         5: 'Car Driver',
         7: 'Car Driver',
         9: 'Car Driver',
         11: 'Car Driver',
         6: 'Car Passenger',
         8: 'Car Passenger',
         10: 'Car Passenger',
         20: 'Car Passenger',
         21: 'Car Passenger',
         12: 'Bus',
         13: 'Bus',
         14: 'Bus',
         15: 'Bus',
         22: 'Bus',
         16: 'Rail',
         17: 'Rail',
         18: 'Rail',
         19: 'Air'}

In [17]:
purposes = {1: 'Commute',
  2: 'Business',
  3: 'Education',
  4: 'Shopping',
  5: 'PB',
  6: 'Social',
  7: 'Visit Friends',
  8: 'Holiday - Day trip',
  11: 'Commute',
  12: 'Business',
  13: 'Education',
  14: 'Escort shopping'}

In [18]:
area_types = {1: 'Urban',
  2: 'Urban',
  3: 'Urban',
  4: 'Urban',
  5: 'Urban',
  6: 'Urban',
  7: 'Urban',
  8: 'Rural'}

In [19]:
other_dic = {0.0: 'other',
 'Shopping': 'other',
 'Social': 'other',
 'Visit Friends': 'other',
 'Holiday - Day trip': 'other',
 'Escort commute': '',
 'Escort business': 'other',
 'Escort shopping': 'other'}

In [25]:
# v10['tfn_at'].replace(area_types, inplace=True)
# v10['trip_purpose'].replace(purposes, inplace=True)
v10['trip_purpose'].replace(other_dic, inplace=True)
# v10['MainMode_B11ID'].replace(modes, inplace=True)

In [31]:
v10.groupby('MainMode_B11ID')['weighted_trips'].sum()

MainMode_B11ID
Air              8.867291e+02
Bus              3.848955e+05
Car Driver       2.561140e+06
Car Passenger    1.406241e+06
Cycle            9.674463e+04
Rail             1.788854e+05
Walk             1.527970e+06
Name: weighted_trips, dtype: float64

In [34]:
v10[v10['MainMode_B11ID'].str.contains('Car', na=False)].groupby('occupant')['weighted_trips'].sum()

occupant
driver       2.561140e+06
passenger    1.406241e+06
Name: weighted_trips, dtype: float64

In [26]:
table_1 = v10.groupby(['tfn_at','trip_purpose','dist_bands','MainMode_B11ID']).sum()['weighted_trips']

C:\Users\Predator\AppData\Local\Temp\ipykernel_2064\4040621858.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  table_1 = v10.groupby(['tfn_at','trip_purpose','dist_bands','MainMode_B11ID']).sum()['weighted_trips']


In [27]:
table_2 = v10[v10['MainMode_B11ID'].str.contains('Car', na=False)].groupby(['tfn_at','trip_purpose','dist_bands','occupant']).sum()['weighted_trips'].unstack()

C:\Users\Predator\AppData\Local\Temp\ipykernel_2064\1021572206.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  table_2 = v10[v10['MainMode_B11ID'].str.contains('Car', na=False)].groupby(['tfn_at','trip_purpose','dist_bands','occupant']).sum()['weighted_trips'].unstack()


In [55]:
v10[v10['tfn_at'].isna()]['HHoldAreaType2_B01ID']

2737087     2
2737088     2
2737089     2
2737090     2
2737091     2
           ..
3009290    10
3009291    10
3009292    10
3009293    10
3009294    10
Name: HHoldAreaType2_B01ID, Length: 272208, dtype: int64

In [52]:
v10[v10['main_mode'].str.contains('Car', na=False)].groupby(['trip_purpose']).sum()['weighted_trips'].sum()

C:\Users\Predator\AppData\Local\Temp\ipykernel_8224\3812257388.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  v10[v10['main_mode'].str.contains('Car', na=False)].groupby(['trip_purpose']).sum()['weighted_trips'].sum()


3923861.6367033673

In [29]:
table_2['occupants'] = (table_2['driver'] + table_2['passenger']) / table_2['driver']

In [28]:
table_1

tfn_at  trip_purpose  dist_bands  MainMode_B11ID
Rural   Business      0-10        Air                   0.000000
                                  Bus                 144.869523
                                  Car Driver        18172.601598
                                  Car Passenger      1895.793023
                                  Cycle               244.239928
                                                        ...     
Urban   other         200+        Car Driver         1636.400162
                                  Car Passenger      1825.130432
                                  Cycle                 0.000000
                                  Rail               1014.988500
                                  Walk                  0.000000
Name: weighted_trips, Length: 420, dtype: float64

In [30]:
table_2

occupant                               driver      passenger  occupants
tfn_at trip_purpose dist_bands                                         
Rural  Business     0-10         18172.601598    1895.793023   1.104321
                    '10-25        8505.172786     707.413100   1.083174
                    25-50         4160.126014     427.378691   1.102732
                    50-100        2356.322322     208.223932   1.088368
                    100-200       1105.673602     119.104184   1.107721
                    200+           204.629008      24.783408   1.121114
       Commute      0-10         92142.177173   15408.501570   1.167225
                    '10-25       40199.336717    3714.041021   1.092391
                    25-50        10273.021644     854.783222   1.083207
                    50-100        2168.411925     162.376299   1.074883
                    100-200        251.972257      30.527844   1.121156
                    200+            30.398667       0.955308   1.031426
       Education    0-10         31259.424048   27468.490296   1.878727
                    '10-25        4274.582452    2447.265442   1.572516
                    25-50          765.881114     223.976977   1.292444
                    50-100         192.743239      45.999109   1.238655
                    100-200         72.413094      21.415710   1.295744
                    200+             9.606732       2.335720   1.243134
       PB           0-10         44212.930716   23324.977991   1.527560
                    '10-25        8345.779643    4620.374178   1.553618
                    25-50         1657.867643     984.858811   1.594052
                    50-100         437.628419     286.596931   1.654886
                    100-200        138.073230      71.046529   1.514557
                    200+            21.668671      23.259705   2.073426
       other        0-10        242023.723614  141603.340593   1.585080
                    '10-25       54058.980549   41129.043972   1.760818
                    25-50        12228.362681   10822.983177   1.885072
                    50-100        4918.995160    4665.215782   1.948408
                    100-200       2533.231217    2404.719983   1.949270
                    200+           603.868799     657.140061   2.088217
Urban  Business     0-10         55225.868565    7437.234427   1.134669
                    '10-25       18305.984871    2139.188785   1.116857
                    25-50         9207.615555    1025.859572   1.111414
                    50-100        5503.157489     722.243556   1.131242
                    100-200       2534.175583     347.528971   1.137137
                    200+           477.000036      79.138565   1.165909
       Commute      0-10        368273.885638   76539.044291   1.207832
                    '10-25       94702.358714   10477.484716   1.110636
                    25-50        24307.213229    2871.453705   1.118132
                    50-100        4685.061621     812.827786   1.173494
                    100-200        668.159554     101.583819   1.152035
                    200+            83.814739      23.212755   1.276953
       Education    0-10        110729.468627  114318.242882   2.032410
                    '10-25        5864.164557    3252.075237   1.554568
                    25-50         1190.118272     446.438437   1.375121
                    50-100         441.796607     185.182273   1.419157
                    100-200        202.588191      87.139509   1.430131
                    200+            34.650658      12.760580   1.368264
       PB           0-10        135079.018797   96178.583590   1.712017
                    '10-25       11701.811512    7820.924478   1.668352
                    25-50         2715.931566    1823.823982   1.671528
                    50-100         964.820993     839.822969   1.870444
                    100-200        371.598810     354.760214   1.954686
                    200+            77.3

In [41]:
table_2.to_csv(r"I:\NTS\outputs\city_science_tables\table_2_combi_edu.csv")

In [35]:
table_1.to_csv(r"I:\NTS\outputs\city_science_tables\table_1_v4.csv")

In [47]:
table_2['driver'].sum() + table_2['passenger'].sum()

3704779.3967269473